# Data Analytics 111-2 Homework #07
Alexander Nilsson

# 1. LR, kNN and SVM on the ORL face dataset

In [5]:
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector
import os


load the data 

In [6]:
# Create an empty NumPy array to store the images and a gender column
#400 rows one for each picture, 2576 columns for the image, 1 for the gender
data_matrix = np.zeros((400, 2577))

# Get a list of all PNG files in the directory
files = glob.glob("ORL Faces/*.png")



# Loop through the image files and read them into the data matrix
for i, filename in enumerate(files):
    # Read the image using OpenCV
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    
    # Resize the image to 46x56
    img = cv2.resize(img, (56, 46))
    
    # Flatten the image into a 1D array and store it in the data matrix
    data_matrix[i, :-1] = img.flatten()
    
    # updt
    # Check the filename to determine gender
    if os.path.basename(filename).startswith(("1_", "8_", "10_", "32_")):
        gender = 0
    else:
        gender = 1
    data_matrix[i, -1] = gender

# Print the shape of the data matrix
print(data_matrix.shape)

(400, 2577)


## Logistic Regression

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_matrix[:, :-1], data_matrix[:, -1], test_size=0.2, random_state=42)

# Create the logistic regression model and fit it to the training data
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

# Evaluate the model on the testing data
lr_score = lr_model.score(X_test, y_test)
print("Logistic Regression accuracy: {:.2f}%".format(lr_score*100))


Logistic Regression accuracy: 97.50%


C:\Users\Aleni\miniconda3\envs\gpu_notebook\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 𝑘-Nearest Neighbors

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_matrix[:, :-1], data_matrix[:, -1], test_size=0.2, random_state=42)

# Fit the KNN model to the training data
k = 5  # number of neighbors to consider
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = knn.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.975


## Support Vector Machine 

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_matrix[:, :-1], data_matrix[:, -1], test_size=0.2, random_state=42)

# Fit the SVM model to the training data
svm = SVC(kernel='linear', C=1)
svm.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = svm.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.975


# 2. The parsimonious principle

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_matrix[:, :-1], data_matrix[:, -1], test_size=0.2, random_state=42)

# Perform feature selection to identify the top 10% of pixels with the highest F-scores
selector = SelectKBest(f_classif, k=int(X_train.shape[1] * 0.1))
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Fit and evaluate the LR model using the selected features
lr = LogisticRegression(random_state=42)
lr.fit(X_train_selected, y_train)
y_pred_lr = lr.predict(X_test_selected)
accuracy_lr = accuracy_score(y_test, y_pred_lr)

# Fit and evaluate the kNN model using the selected features
k = 5  # number of neighbors to consider
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train_selected, y_train)
y_pred_knn = knn.predict(X_test_selected)
accuracy_knn = accuracy_score(y_test, y_pred_knn)

# Fit and evaluate the SVM model using the selected features
svm = SVC(kernel='linear', C=1, random_state=42)
svm.fit(X_train_selected, y_train)
y_pred_svm = svm.predict(X_test_selected)
accuracy_svm = accuracy_score(y_test, y_pred_svm)

# Print the accuracies of the three models
print("LR accuracy:", accuracy_lr)
print("kNN accuracy:", accuracy_knn)
print("SVM accuracy:", accuracy_svm)

LR accuracy: 0.9125
kNN accuracy: 0.9625
SVM accuracy: 0.925


C:\Users\Aleni\miniconda3\envs\gpu_notebook\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
# Define the array of values
values = [0.1, 0.05, 0.01, 0.001]

# Print the header for the table
print("{:<10} {:<15} {:<15} {:<15}".format("% Parameters", "LR accuracy", "kNN accuracy", "SVM accuracy"))

# Loop over the p values and fit/evaluate the models
for p in values:
    # Perform feature selection to identify the top p% of pixels with the highest F-scores
    selector = SelectKBest(f_classif, k=int(X_train.shape[1] * p))
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    # Fit and evaluate the LR model using the selected features
    lr = LogisticRegression(random_state=42)
    lr.fit(X_train_selected, y_train)
    y_pred_lr = lr.predict(X_test_selected)
    accuracy_lr = accuracy_score(y_test, y_pred_lr)

    # Fit and evaluate the kNN model using the selected features
    k = 5  # number of neighbors to consider
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_selected, y_train)
    y_pred_knn = knn.predict(X_test_selected)
    accuracy_knn = accuracy_score(y_test, y_pred_knn)

    # Fit and evaluate the SVM model using the selected features
    svm = SVC(kernel='linear', C=1, random_state=42)
    svm.fit(X_train_selected, y_train)
    y_pred_svm = svm.predict(X_test_selected)
    accuracy_svm = accuracy_score(y_test, y_pred_svm)

    # Print the accuracies of the three models for the current p value
    print("{:<10} {:<15.4f} {:<15.4f} {:<15.4f}".format(p, accuracy_lr, accuracy_knn, accuracy_svm))

% Parameters LR accuracy     kNN accuracy    SVM accuracy   
0.1        0.9125          0.9625          0.9250         
0.05       0.9250          0.9625          0.9000         


C:\Users\Aleni\miniconda3\envs\gpu_notebook\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Aleni\miniconda3\envs\gpu_notebook\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

0.01       0.9000          0.9500          0.9125         
0.001      0.9250          0.9500          0.9250         


Regarding the results of the model evaluation, the table shows the accuracy scores of the three models for different percentages of features selected. As the percentage of features selected decreases from 0.1 to 0.001, the accuracy scores of all three models generally decrease as well. This suggests that selecting a smaller number of features may lead to a loss of important information for the classification task, resulting in lower accuracy.

The k-Nearest Neighbors model generally has the highest accuracy scores, followed by the Logistic Regression and Support Vector Machines models. However, the differences in accuracy between the models are relatively small, especially for higher percentages of features selected (0.1 and 0.05), indicating that all three models perform reasonably well on this classification task.

Overall, the results suggest that using feature selection to select the top p% of features with the highest F-scores can improve the performance of the classification models. However, the optimal percentage of features to select may depend on the specific dataset and classification task.

# 3. AutoMPG classify Orgin

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# define the column names
columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin', 'name']

# read the data from the text file
auto = pd.read_csv('auto-mpg.data.txt', delim_whitespace=True, names=columns, na_values='?')

auto.drop('name', axis=1, inplace=True)
auto.dropna(inplace=True)

# Preprocess the data
X = auto[['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year']]
y = auto['origin']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit and evaluate the LR model
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=42)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
report_lr = classification_report(y_test, y_pred_lr)

# Fit and evaluate the kNN model
k = 5  # number of neighbors to consider
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
report_knn = classification_report(y_test, y_pred_knn)

# Fit and evaluate the SVM model
svm = SVC(kernel='linear', C=1, random_state=42)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
report_svm = classification_report(y_test, y_pred_svm)

# Print the accuracies and classification reports of the three models
print("LR accuracy:", accuracy_lr)
print(report_lr)
print("kNN accuracy:", accuracy_knn)
print(report_knn)
print("SVM accuracy:", accuracy_svm)
print(report_svm)


C:\Users\Aleni\miniconda3\envs\gpu_notebook\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR accuracy: 0.7088607594936709
              precision    recall  f1-score   support

           1       0.86      0.88      0.87        43
           2       0.67      0.30      0.41        20
           3       0.46      0.75      0.57        16

    accuracy                           0.71        79
   macro avg       0.66      0.64      0.62        79
weighted avg       0.73      0.71      0.70        79

kNN accuracy: 0.6582278481012658
              precision    recall  f1-score   support

           1       0.72      0.95      0.82        43
           2       0.62      0.25      0.36        20
           3       0.43      0.38      0.40        16

    accuracy                           0.66        79
   macro avg       0.59      0.53      0.53        79
weighted avg       0.64      0.66      0.62        79

SVM accuracy: 0.7341772151898734
              precision    recall  f1-score   support

           1       0.84      0.88      0.86        43
           2       0.61      0.

The accuracy scores of the three models are in the range of 0.65 to 0.73, indicating that they perform reasonably well on this classification task. 

The Support Vector Machines model has the highest accuracy score (0.734), followed by the Logistic Regression model (0.709) and k-Nearest Neighbors model (0.658). 

The classification reports show that the performance of the models varies across the different class labels. For example, the Logistic Regression model performs well on origin 1 (precision=0.86, recall=0.88), but poorly on origin 2 (precision=0.67, recall=0.30), indicating that it has more difficulty in correctly identifying cars from origin 2 than from origin 1. 

The k-Nearest Neighbors model performs well on origin 1 (precision=0.72, recall=0.95), but poorly on origin 2 (precision=0.62, recall=0.25) and origin 3 (precision=0.43, recall=0.38), suggesting that it struggles to distinguish between these two classes. 

The Support Vector Machines model performs well on origin 1 (precision=0.84, recall=0.88) and origin 2 (precision=0.61, recall=0.70), but less well on origin 3 (precision=0.55, recall=0.38).

Overall, the results suggest that the three models have different strengths and weaknesses in classifying cars by origin. The Support Vector Machines model is the most accurate overall, but has some difficulty in correctly classifying cars from origin 3. The Logistic Regression model performs well on origin 1, but struggles with origin 2. The k-Nearest Neighbors model has good recall for origin 1, but poor precision and recall for origins 2 and 3. 